In [1]:
import pandas as pd

In [2]:
folder = '.\\input_files\\'
file = '221487576_southern-ocean-medical-center_standardcharges.xlsx'

In [3]:
if file == '221487576_raritan-bay-medical-center_standardcharges.xlsx':
    df = pd.read_excel(folder + file, dtype=str, skiprows=4)
else:
    df = pd.read_excel(folder + file, dtype=str, skiprows=3)

In [4]:
df1 = df.copy()

In [5]:
df = df1.copy()

In [6]:
df.rename(columns=lambda x: x.strip().replace('\n', ' '), inplace=True)
df.rename(columns={
    'Service Area': 'setting',
    'Payor': 'payer_name',
    'MSDRG': 'ms_drg',
    'MSDRG Name': 'description',
    'Px Code': 'local_code',
    'Px Description': 'description1',
    'Primary/Bundled CPT Code': 'hcpcs_cpt',
    'CPT/HCPCS Code': 'alt_hcpcs_cpt',
    'Revenue Code': 'rev_code',
    'Service_x000D_ Area': 'setting'
}, inplace=True)

In [7]:
df.drop('Location', axis=1, inplace=True)

In [8]:
df.loc[df['local_code'] == 'See MSDRG', 'local_code'] = df['ms_drg']
df.loc[df['description1'] == 'See MSDRG Name', 'description1'] = pd.NA
df.loc[(~df['description1'].isna()) &  (df['description'].isna()), 'description'] = df['description1']

In [9]:
df.drop('description1', axis=1, inplace=True)

In [10]:
df

,setting,payer_name,ms_drg,description,local_code,hcpcs_cpt,alt_hcpcs_cpt,rev_code,Gross Charge,Discounted Cash Price,Payor Specific Negotiated Charge,De-Identified Minimum,De-Identified Maximum
0,Inpatient,Aetna,246,Percutaneous cardiovascular procedures with dr...,246,NaN,NaN,NaN,131644.73227848098,NaN,80832.44346542846,11215.5235391286,111898.02243670882
1,Inpatient,Aetna,247,Percutaneous cardiovascular procedures with dr...,247,NaN,NaN,NaN,81165.65242937856,NaN,50758.98892968721,6918,68990.80456497178
2,Inpatient,Aetna,331,Major small and large bowel procedures without...,331,NaN,NaN,NaN,68181.78399999999,NaN,43679.50724481741,6140.304392561364,57954.516399999986
3,Inpatient,Aetna,460,Spinal fusion except cervical without MCC,460,NaN,NaN,NaN,177110.095,NaN,103392.67367194849,10115,150543.58075
4,Inpatient,Aetna,468,Revision of hip or knee replacement without CC...,468,NaN,NaN,NaN,69040.75333333333,NaN,65259.787869597,6069,84142.06000000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
237341,All,NaN,NaN,WAND COBLATION HALO,5010000272,NaN,NaN,272,674,NaN,NaN,NaN,NaN
237342,All,NaN,NaN,PLATE BNT GRV PRBNT 35X16.5X2.5MM TI 4/4.5MM S...,501C171301,NaN,C1713,278,2500,NaN,NaN,NaN,NaN
237343,All,NaN,NaN,GRAFT VASC GORE PROPATEN 80CM,501C176801,NaN,C1768,278,4656,NaN,NaN,NaN,NaN
237344,All,NaN,NaN,CAGE CONCORGE PROTI P 9X11X27MM,501C188901,NaN,C1889,278,8000,NaN,NaN,NaN,NaN


In [11]:
df['setting'] = df['setting'].str.lower()
df.loc[df['setting'] == 'all', 'setting'] = 'both'

In [12]:
df['ms_drg'] = df['ms_drg'].str.zfill(3)

In [13]:
df_payer = df.loc[(~df['payer_name'].isna())].copy()

In [14]:
df_payer.drop(['Gross Charge', 'Discounted Cash Price', 'De-Identified Minimum', 'De-Identified Maximum'], axis=1, inplace=True)

In [15]:
df_payer.rename(columns={'Payor Specific Negotiated Charge': 'standard_charge'}, inplace=True)

In [16]:
df_payer.dropna(subset=['standard_charge'], inplace=True)

In [17]:
df_payer['payer_category'] = 'payer'

In [18]:
df.drop(['payer_name', 'Payor Specific Negotiated Charge'], axis=1, inplace=True)

In [19]:
cols = df.columns.tolist()
id_vars = cols[:7]
value_vars = cols[7:]


df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

In [20]:
mapping = {
    'Gross Charge': 'gross',
    'Discounted Cash Price': 'cash',
    
    'De-Identified Minimum': 'min',
    'De-Identified Maximum': 'max'    
}

df['payer_category'] = df['payer_name'].map(mapping)

In [21]:
df

,setting,ms_drg,description,local_code,hcpcs_cpt,alt_hcpcs_cpt,rev_code,payer_name,standard_charge,payer_category
0,inpatient,246,Percutaneous cardiovascular procedures with dr...,246,NaN,NaN,NaN,Gross Charge,131644.73227848098,gross
1,inpatient,247,Percutaneous cardiovascular procedures with dr...,247,NaN,NaN,NaN,Gross Charge,81165.65242937856,gross
2,inpatient,331,Major small and large bowel procedures without...,331,NaN,NaN,NaN,Gross Charge,68181.78399999999,gross
3,inpatient,460,Spinal fusion except cervical without MCC,460,NaN,NaN,NaN,Gross Charge,177110.095,gross
4,inpatient,468,Revision of hip or knee replacement without CC...,468,NaN,NaN,NaN,Gross Charge,69040.75333333333,gross
...,...,...,...,...,...,...,...,...,...,...
949379,both,NaN,WAND COBLATION HALO,5010000272,NaN,NaN,272,De-Identified Maximum,NaN,max
949380,both,NaN,PLATE BNT GRV PRBNT 35X16.5X2.5MM TI 4/4.5MM S...,501C171301,NaN,C1713,278,De-Identified Maximum,NaN,max
949381,both,NaN,GRAFT VASC GORE PROPATEN 80CM,501C176801,NaN,C1768,278,De-Identified Maximum,NaN,max
949382,both,NaN,CAGE CONCORGE PROTI P 9X11X27MM,501C188901,NaN,C1889,278,De-Identified Maximum,NaN,max


In [22]:
df = pd.concat([df, df_payer])

In [23]:
df['rev_code'] = df['rev_code'].str.zfill(4)

In [24]:
df.reset_index(inplace=True, drop=True)
mask = df['alt_hcpcs_cpt'].str.len() > 5
df.loc[mask, 'code'] = df['alt_hcpcs_cpt']
df.loc[mask, 'alt_hcpcs_cpt'] = pd.NA

mask = df['hcpcs_cpt'].str.len() > 5
df.loc[mask, 'code'] = df['hcpcs_cpt']
df.loc[mask, 'hcpcs_cpt'] = pd.NA

In [25]:
df.loc[~df['code'].isna()]

,setting,ms_drg,description,local_code,hcpcs_cpt,alt_hcpcs_cpt,rev_code,payer_name,standard_charge,payer_category,code
133114,both,NaN,TELEMETRY ADULT,1000000034,NaN,NaN,0206,Gross Charge,8600,gross,1000000034
133115,both,NaN,TELEMETRY ADULT,1000000034,NaN,NaN,0206,Gross Charge,8600,gross,1000000034
133116,both,NaN,TELEMETRY ADULT,1000000034,NaN,NaN,0206,Gross Charge,8600,gross,1000000034
133117,both,NaN,TELEMETRY PEDIATRIC,1000000036,NaN,NaN,0206,Gross Charge,10200,gross,1000000036
133118,both,NaN,TELEMETRY PEDIATRIC,1000000036,NaN,NaN,0206,Gross Charge,10200,gross,1000000036
...,...,...,...,...,...,...,...,...,...,...,...
910586,both,NaN,TISSUE ALLOGRAFT AMNION AMNIO-MAXX CRYO 1ML,6010010635,NaN,NaN,0278,De-Identified Maximum,NaN,max,6010010635
910587,both,NaN,TISSUE ALLOGRAFT AMNION AMNIO-MAXX CRYO 1ML,6010010635,NaN,NaN,0278,De-Identified Maximum,NaN,max,6010010635
914932,both,NaN,STENT WEB 17 7X5,6010012116,NaN,NaN,0278,De-Identified Maximum,NaN,max,6010012116
914933,both,NaN,STENT WEB 17 7X5,6010012116,NaN,NaN,0278,De-Identified Maximum,NaN,max,6010012116


In [26]:
mapping = {'221487576_bayshore-medical-center_standardcharges.xlsx': '310112',
 '221487576_hackensack-university-medical-center_standardcharges.xlsx': '310001',
 '221487576_jersey-shore-university-medical-center_standardcharges.xlsx': '310073',
 '221487576_jfk-university-medical-center_standardcharges.xlsx': '310108',
 '221487576_ocean-medical-center_standardcharges.xlsx': '310052',
 '221487576_palisades-medical-center_standardcharges.xlsx': '310003',
 '221487576_raritan-bay-medical-center_standardcharges.xlsx': '310039',
 '221487576_riverview-medical-center_standardcharges.xlsx': '310034',
 '221487576_southern-ocean-medical-center_standardcharges.xlsx': '310113'}

hospital_id = mapping[file]

df['hospital_id'] = hospital_id

In [30]:
df

,setting,ms_drg,description,local_code,hcpcs_cpt,alt_hcpcs_cpt,rev_code,payer_name,standard_charge,payer_category,code,hospital_id
0,inpatient,246,Percutaneous cardiovascular procedures with dr...,246,NaN,NaN,NaN,Gross Charge,131644.73227848098,gross,NaN,310113
1,inpatient,247,Percutaneous cardiovascular procedures with dr...,247,NaN,NaN,NaN,Gross Charge,81165.65242937856,gross,NaN,310113
2,inpatient,331,Major small and large bowel procedures without...,331,NaN,NaN,NaN,Gross Charge,68181.78399999999,gross,NaN,310113
3,inpatient,460,Spinal fusion except cervical without MCC,460,NaN,NaN,NaN,Gross Charge,177110.095,gross,NaN,310113
4,inpatient,468,Revision of hip or knee replacement without CC...,468,NaN,NaN,NaN,Gross Charge,69040.75333333333,gross,NaN,310113
...,...,...,...,...,...,...,...,...,...,...,...,...
1068544,outpatient,NaN,PROPOFOL 10 MG/ML EMUL,7000000005,G0105,J2704,0636,Wellcare Medicare,29.25,payer,NaN,310113
1068545,outpatient,NaN,DRUG 250,7000000007,66984,NaN,0250,Wellcare Medicare,0,payer,NaN,310113
1068546,outpatient,NaN,DRUG 250,7000000010,92611,NaN,0250,Wellcare Medicare,0,payer,NaN,310113
1068547,outpatient,NaN,DRUG 250,DRUG 250,G0105,NaN,0250,Wellcare Medicare,0,payer,NaN,310113


In [31]:
df.drop_duplicates(subset=['setting', 'ms_drg', 'description', 'local_code', 'hcpcs_cpt', 'alt_hcpcs_cpt', 'rev_code', 'payer_name', 'payer_category', 'code'])

,setting,ms_drg,description,local_code,hcpcs_cpt,alt_hcpcs_cpt,rev_code,payer_name,standard_charge,payer_category,code,hospital_id
0,inpatient,246,Percutaneous cardiovascular procedures with dr...,246,NaN,NaN,NaN,Gross Charge,131644.73227848098,gross,NaN,310113
1,inpatient,247,Percutaneous cardiovascular procedures with dr...,247,NaN,NaN,NaN,Gross Charge,81165.65242937856,gross,NaN,310113
2,inpatient,331,Major small and large bowel procedures without...,331,NaN,NaN,NaN,Gross Charge,68181.78399999999,gross,NaN,310113
3,inpatient,460,Spinal fusion except cervical without MCC,460,NaN,NaN,NaN,Gross Charge,177110.095,gross,NaN,310113
4,inpatient,468,Revision of hip or knee replacement without CC...,468,NaN,NaN,NaN,Gross Charge,69040.75333333333,gross,NaN,310113
...,...,...,...,...,...,...,...,...,...,...,...,...
1068544,outpatient,NaN,PROPOFOL 10 MG/ML EMUL,7000000005,G0105,J2704,0636,Wellcare Medicare,29.25,payer,NaN,310113
1068545,outpatient,NaN,DRUG 250,7000000007,66984,NaN,0250,Wellcare Medicare,0,payer,NaN,310113
1068546,outpatient,NaN,DRUG 250,7000000010,92611,NaN,0250,Wellcare Medicare,0,payer,NaN,310113
1068547,outpatient,NaN,DRUG 250,DRUG 250,G0105,NaN,0250,Wellcare Medicare,0,payer,NaN,310113


In [32]:
df.to_csv('test.csv', index=False)

In [27]:
folder = '.\\output_files\\'
# file = hospital_id + file.split('_')[1] + '.csv'

# df.to_csv('test.csv', index=False)
# df.to_csv(folder + file, index=False)